#### To do:
- get names from file 
- access dbpedia query 
- extract information

#### To do after:
- info extraction from comments 
-- maybe only when infomation is missing

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import rdflib

In [2]:
g = rdflib.Graph().parse('thesisKG.ttl', format='ttl')

In [ ]:
d=pd.DataFrame(g)
d.rename(columns={0: 'S', 1: 'P', 2: 'O'}, inplace=True)
s = []
s_source = []
for i in d.S:
    topic = i.split('/')[-1]
    s_source.append(topic)
    split = topic.split('_')
    s.append(' '.join(split))
d['label'] = s
d['dboSource'] = s_source
o = []
for i in d.O:
    topic = i.split('/')[-1]
    split = topic.split('_')
    o.append(' '.join(split))
d['O'] = o
d

,S,P,O,label,dboSource
0,http://yago-knowledge.org/resource/Fountain,http://www.w3.org/2000/01/rdf-schema#label,Vodna fontana,Fountain,Fountain
1,http://yago-knowledge.org/resource/Darrin_Hancock,http://schema.org/birthPlace,"Birmingham, Alabama",Darrin Hancock,Darrin_Hancock
2,http://yago-knowledge.org/resource/Conflict_(p...,http://www.w3.org/2000/01/rdf-schema#label,konflikt,Conflict (process),Conflict_(process)
3,http://yago-knowledge.org/resource/Flood,http://www.w3.org/2000/01/rdf-schema#label,Üleujotus,Flood,Flood
4,http://yago-knowledge.org/resource/Cryoshell_(...,http://schema.org/genre,Symphonic rock,Cryoshell (album),Cryoshell_(album)
...,...,...,...,...,...
3152363,http://yago-knowledge.org/resource/Mike_Torchia,http://schema.org/memberOf,Dallas Stars,Mike Torchia,Mike_Torchia
3152364,http://yago-knowledge.org/resource/Constellation,http://www.w3.org/2000/01/rdf-schema#label,stjernebillede,Constellation,Constellation
3152365,http://yago-knowledge.org/resource/Oonche_Log_...,http://schema.org/actor,Rajesh Khanna,Oonche Log (1985 film),Oonche_Log_(1985_film)
3152366,http://yago-knowledge.org/resource/Broadway_(b...,http://schema.org/foundingLocation,"Orlando, Florida",Broadway (band),Broadway_(band)


In [1]:
pops = d[d.O.isin(['popularEntity'])].reset_index()
pops = pops.drop('index', axis=1)
pops[-1]

NameError: name 'd' is not defined

In [54]:
len(pops)

12763

In [79]:
def query(popent):
    fun = False
    while fun == False:
        try:
            sparql = SPARQLWrapper("http://dbpedia.org/sparql")
            sparql.setReturnFormat(JSON)
            sparql.setQuery("""
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                PREFIX dbo: <http://dbpedia.org/ontology/>
                SELECT ?abstract
                WHERE { 
                ?s rdfs:label "%s" @en .
                ?s dbo:abstract ?abstract .
                FILTER (LANG(?abstract) = "en")
                }
                """ %(popent))
            results = sparql.query().convert()
            fun = True
        except:
            fun = False
    return results 

abstracts = []
j = 0
for label in pops.label:
    if j % 1000 == 0:
        print(j)
    pastablen = len(abstracts)
    results = query(label)
    if len(results["results"]["bindings"]) == 0:
        abstracts.append('')
        j+=1
        continue 
    for result in results["results"]["bindings"]:
        abstracts.append(result['abstract']['value'])
        j += 1
j, len(pops)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


(12763, 12763)

In [80]:
len(pops), len(abstracts), len(pops.label)

(12763, 12763, 12763)

In [86]:
# There are 309 (out of 12763) entities that their abstract could not be accessed 
blanks = 0
for i in abstracts:
    if len(i) == 0:
        blanks += 1
blanks

309

In [84]:
pops['Abstracts']= abstracts
pops

In [85]:
pops.to_csv('abstracts.csv')